# Silver → Gold Data Quality Classification

> Gold is mainly a **semantic/modeling layer** (star schema + conformance).  
> Most “row-level cleansing” happens in Silver.
> Gold focuses on **integration, conformed dimensions, and analytic usability**.

## `gold.dim_customers` 

### DQ dimensions addressed / reinforced
- **Completeness**: enriches with country/birthdate; uses ERP gender as fallback when CRM gender is `'n/a'`.
- **Consistency**: one standardized customer view across CRM + ERP with consistent column naming.
- **Validity**: relies on Silver’s controlled-domain values (`Male/Female/n/a`) and cleaned join keys.
- **Accuracy**: “preferred source” rule for gender improves semantic correctness when CRM is unknown.
- **Uniqueness**: introduces a surrogate `customer_key` for star schema joins.
  - Note: `ROW_NUMBER()` is unique per run but not stable if ordering changes (key stability concern).

### Practical impact
Analysts get one authoritative customer dimension with integrated demographics and location attributes.

## `gold.dim_products` — Conformed product dimension (current version)


### DQ dimensions addressed / reinforced
- **Timeliness**: selects only current product versions (`prd_end_dt IS NULL`).
- **Completeness**: enriches products with category/subcategory/maintenance (mapping coverage may still be incomplete).
- **Consistency**: conforms product and category attributes into a single standardized view.
- **Uniqueness**: introduces `product_key` as a surrogate key.

### Practical impact
A clean, analytics-friendly product dimension including category hierarchy and only active products.

## `gold.fact_sales`

### DQ dimensions addressed / reinforced
- **Consistency**: standardizes field naming and aligns the fact with conformed dimensions.
- **Validity**: enforces referential alignment (missing lookups show up as NULL dimension keys).
- **Completeness**: makes dimensional coverage measurable (NULL `product_key` / `customer_key` indicates gaps).
- **Accuracy**: reporting uses standardized dimension attributes rather than raw identifiers.

### Practical impact
A BI-ready fact view that follows star schema join patterns.

## Summary: Silver → Gold changes and DQ impact

| Gold object | What Gold is doing | DQ dimensions impacted |
|---|---|---|
| `gold.dim_customers` | Integrates customer master with ERP gender/birthdate and location; generates surrogate `customer_key`; chooses preferred gender source | **Completeness**, **Consistency**, **Validity**, **Accuracy** (light), **Uniqueness** (structural) |
| `gold.dim_products` | Conforms products with category attributes; generates surrogate `product_key`; filters to current product versions (`prd_end_dt IS NULL`) | **Consistency**, **Timeliness**, **Completeness** (category enrichment), **Uniqueness** (structural) |
| `gold.fact_sales` | Conforms sales facts by attaching `product_key` and `customer_key` via joins; standardizes column names and grain for analytics | **Consistency**, **Validity** (FK alignment), **Completeness** (FK coverage), **Accuracy** (conformed lookups) |